## Image Stats

By default we were normalizing with ImageNet stats. Obviously these don't apply to our dataset so let's try calculating the means and standard deviations for our stats.

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
import numpy
import cv2
import numpy as np
from tqdm import tqdm

In [2]:
path = '/home/josh/git/AudioTagging/work/image/trn_merged2/'
train_images = os.listdir(path)
train_images = [path + s for s in train_images]

In [3]:
train_images[0]

'/home/josh/git/AudioTagging/work/image/trn_merged2/b157987e.wav.jpg'

In [4]:
def getMeans(paths):
    redSum = 0
    greenSum = 0
    blueSum = 0

    for path in paths:
        image = cv2.imread(path)
        
        red = np.reshape(image[:,:,0], -1)
        green = np.reshape(image[:,:,1], -1)
        blue = np.reshape(image[:,:,2], -1)

        redMean = red.mean()
        greenMean = green.mean()
        blueMean = blue.mean()

        redSum = redSum + redMean
        greenSum = greenSum + greenMean
        blueSum = blueSum + blueMean
        
    redMean = redSum/len(paths)
    greenMean = greenSum/len(paths)
    blueMean = blueSum/len(paths)
    
    return (redMean, greenMean, blueMean)

In [5]:
rawMeans = getMeans(train_images)
redMean, greenMean, blueMean = rawMeans

In [6]:
def getStds(paths, redMean, greenMean, blueMean):
    redSum = 0
    greenSum = 0
    blueSum = 0

    for path in paths:
        image = cv2.imread(path)
        rows, columns, channels = image.shape
        red = np.reshape(image[:,:,0], -1)
        green = np.reshape(image[:,:,1], -1)
        blue = np.reshape(image[:,:,2], -1)

        redDiffs = red - redMean
        redSumOfSquares = np.sum(redDiffs ** 2)
        
        blueDiffs = blue - blueMean
        blueSumOfSquares = np.sum(blueDiffs ** 2)
        
        greenDiffs = green - greenMean
        greenSumOfSquares = np.sum(greenDiffs ** 2)
        
        redSum = redSum + (1/(len(paths) * rows * columns)) * redSumOfSquares
        greenSum = greenSum + (1/(len(paths) * rows * columns)) * greenSumOfSquares
        blueSum = blueSum + (1/(len(paths) * rows * columns)) * blueSumOfSquares
        
    redStd = np.sqrt(redSum)
    greenStd = np.sqrt(greenSum)
    blueStd = np.sqrt(blueSum)
    
    return (redStd, greenStd, blueStd)

In [7]:
redStd, greenStd, blueStd = getStds(train_images, redMean, greenMean, blueMean)

In [8]:
print("Red: ", redMean, redStd)
print("Green: ", greenMean, greenStd)
print("Blue: ", blueMean, blueStd)

Red:  99.18429690025181 59.82032889965118
Green:  99.18429690025181 59.82032889965118
Blue:  99.18429690025181 59.82032889965118


In [9]:
stats = [[redMean, greenMean, blueMean], [redStd, blueStd, greenStd]]
print(stats)

[[99.18429690025181, 99.18429690025181, 99.18429690025181], [59.82032889965118, 59.82032889965118, 59.82032889965118]]


In [10]:
normalized_stats = [[redMean/255, greenMean/255, blueMean/255], [redStd/255, blueStd/255, greenStd/255]]
print(normalized_stats)

[[0.38895802705981103, 0.38895802705981103, 0.38895802705981103], [0.2345895250966713, 0.2345895250966713, 0.2345895250966713]]
